# 畳み込みニューラルネットワーク(CNN)のCIFAR10データセットへの適用

pytorchをインストールします．
CUDAのバージョンは8.0，pytorchのバージョンは0.4.0です．


pytorchに必要なパッケージ類をインポートします

In [ ]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch.nn.functional as F


In [ ]:
torch.cuda.is_available()

True

In [ ]:
num_epochs    = 100
batch_size    = 128
learning_rate = 1e-3
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


In [ ]:
transform_train = transforms.Compose([transforms.RandomCrop(32, padding=4), transforms.RandomHorizontalFlip(), transforms.ToTensor(),  transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),])
transform_test = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),])


In [ ]:
trainset = dsets.CIFAR10(root='./data', train=True,  download=True, transform=transform_train)
testset =  dsets.CIFAR10(root='./data', train=False,download=True, transform=transform_test)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(testset, batch_size=100,shuffle=False, num_workers=2)

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
net = Net()   

In [ ]:
use_cuda = True
if use_cuda and torch.cuda.is_available():
    net.cuda()

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
for epoch in range(num_epochs):
    train_loss = 0.0
    correct = 0
    total = 0
    for batch_idx, (images, labels) in enumerate(train_loader):
        images, labels = images.to("cuda"), labels.to("cuda")

        optimizer.zero_grad()               
        outputs = net(images)              
        loss = criterion(outputs, labels)
        loss.backward()                           
        optimizer.step()                          
        train_loss += loss.data[0]

        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()     
    print( epoch, 'Loss: %.5f | Acc: %.3f%% (%d/%d)' % (train_loss/((len(train_loader) * batch_size)), 100.*correct/total, correct, total))
            

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  del sys.path[0]


0 Loss: 0.00980 | Acc: 55.274% (27637/50000)
1 Loss: 0.00971 | Acc: 55.888% (27944/50000)
2 Loss: 0.00964 | Acc: 55.984% (27992/50000)
3 Loss: 0.00957 | Acc: 56.246% (28123/50000)
4 Loss: 0.00950 | Acc: 56.492% (28246/50000)


Exception ignored in: <bound method Image.__del__ of <PIL.Image.Image image mode=RGB size=40x40 at 0x7F68B0024588>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/PIL/Image.py", line 589, in __del__
Process Process-63:
    if (hasattr(self, 'fp') and hasattr(self, '_exclusive_fp')
KeyboardInterrupt
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 57, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 57, in <listcomp>
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/usr/local/lib/python3.6/dist-packages/torchvision/datasets/cifar.py", line 1

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-15-ddd347de831e>", line 5, in <module>
    for batch_idx, (images, labels) in enumerate(train_loader):
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 280, in __next__
    idx, batch = self._get_batch()
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 259, in _get_batch
    return self.data_queue.get()
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 335, in get
    res = self._reader.recv_bytes()
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 379, 

KeyboardInterrupt: ignored

In [ ]:
net.eval()
test_loss = 0
correct = 0
total = 0
for batch_idx, (inputs, targets) in enumerate(test_loader):
    inputs, targets = inputs.to("cuda"), targets.to("cuda")
    outputs = net(inputs)
    loss = criterion(outputs, targets)

    test_loss += loss.item()
    _, predicted = outputs.max(1)
    total += targets.size(0)
    correct += predicted.eq(targets).sum().item()

print( 'Loss: %.5f | Acc: %.3f%% (%d/%d)' % (test_loss/((len(test_loader)*100)), 100.*correct/total, correct, total))
            

Loss: 0.01133 | Acc: 59.770% (5977/10000)


In [ ]:
torch.save(net.state_dict(), 'fnn_model.pkl')